In [3]:
!pip install textblob
!pip install vaderSentiment

In [4]:
# Cell 1: Text Preprocessing & Feature Engineering Setup
# This cell sets up the preprocessing pipeline for hybrid CNN + contextual features

import pandas as pd
import numpy as np
import re
import string
import nltk
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
import json
from pathlib import Path

# Download NLTK data (run once)
print("🔧 Setting up NLTK resources...")
nltk_downloads = ['punkt', 'stopwords', 'vader_lexicon', 'wordnet', 'averaged_perceptron_tagger']
for resource in nltk_downloads:
    try:
        nltk.data.find(f'tokenizers/{resource}')
        print(f"✅ {resource} already downloaded")
    except LookupError:
        print(f"📥 Downloading {resource}...")
        nltk.download(resource, quiet=True)

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

print("\n🧹 Text Preprocessing & Feature Engineering Pipeline")
print("=" * 55)

# Initialize preprocessing tools
lemmatizer = WordNetLemmatizer()
sentiment_analyzer = SentimentIntensityAnalyzer()
stop_words = set(stopwords.words('english'))

# Load our datasets from previous exploration
print("📂 Loading datasets from data exploration...")
try:
    # Load headlines dataset
    with open('data/raw/Sarcasm_Headlines_Dataset_v2.json', 'r') as f:
        headlines_data = [json.loads(line) for line in f]
    headlines_df = pd.DataFrame(headlines_data)
    
    # Load reddit dataset (sample for processing speed)
    reddit_df = pd.read_csv('data/raw/train-balanced-sarcasm.csv')
    reddit_sample = reddit_df.sample(n=50000, random_state=42)  # 50K sample for training
    
    print(f"✅ Headlines: {len(headlines_df):,} samples")
    print(f"✅ Reddit: {len(reddit_sample):,} samples (sampled)")
    
except Exception as e:
    print(f"❌ Error loading data: {e}")

# Define text cleaning function
def clean_text(text):
    """
    Advanced text cleaning for sarcasm detection
    Preserves some punctuation patterns important for sarcasm
    """
    if pd.isna(text):
        return ""
    
    # Convert to string and lowercase
    text = str(text).lower()
    
    # Remove URLs but keep the structure for sarcasm detection
    text = re.sub(r'http\S+|www\S+', '[URL]', text)
    
    # Handle contractions (keep some for sarcasm patterns)
    contractions = {
        "won't": "will not", "can't": "cannot", "n't": " not",
        "'re": " are", "'ve": " have", "'ll": " will", "'d": " would"
    }
    for contraction, expansion in contractions.items():
        text = text.replace(contraction, expansion)
    
    # Remove extra whitespace but preserve sentence structure
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove excessive punctuation but keep some for sarcasm detection
    text = re.sub(r'[!]{2,}', '!!', text)  # Max 2 exclamation marks
    text = re.sub(r'[?]{2,}', '??', text)  # Max 2 question marks
    text = re.sub(r'[.]{3,}', '...', text)  # Standardize ellipsis
    
    return text

print("\n🎯 Preprocessing Functions Defined:")
print("  ✅ Text cleaning (preserves sarcasm patterns)")
print("  ✅ NLTK tools initialized") 
print("  ✅ Sentiment analyzer ready")
print("  ✅ Datasets loaded and ready")

print(f"\n📋 Next Steps:")
print("1. Apply text cleaning to both datasets")
print("2. Extract contextual features (sentiment, incongruity)")
print("3. Create hybrid feature vectors")
print("4. Split data for training/validation/testing")

🔧 Setting up NLTK resources...
✅ punkt already downloaded
📥 Downloading stopwords...
📥 Downloading vader_lexicon...
📥 Downloading wordnet...
📥 Downloading averaged_perceptron_tagger...

🧹 Text Preprocessing & Feature Engineering Pipeline
📂 Loading datasets from data exploration...


✅ Headlines: 28,619 samples
✅ Reddit: 50,000 samples (sampled)

🎯 Preprocessing Functions Defined:
  ✅ Text cleaning (preserves sarcasm patterns)
  ✅ NLTK tools initialized
  ✅ Sentiment analyzer ready
  ✅ Datasets loaded and ready

📋 Next Steps:
1. Apply text cleaning to both datasets
2. Extract contextual features (sentiment, incongruity)
3. Create hybrid feature vectors
4. Split data for training/validation/testing


In [5]:
# Cell 2: Text Cleaning & Contextual Feature Extraction
# Extract linguistic features: sentiment polarity variation, incongruity, hyperbole patterns

from collections import Counter
import math

print("🧹 Applying Text Cleaning & Feature Extraction...")
print("=" * 50)

def extract_contextual_features(text):
    """
    Extract contextual features for sarcasm detection as per research abstract:
    - Sentiment polarity variation
    - Incongruity patterns  
    - Hyperbole detection
    - Lexical pattern signals
    """
    features = {}
    
    # 1. SENTIMENT POLARITY VARIATION
    sentiment_scores = sentiment_analyzer.polarity_scores(text)
    features['sentiment_compound'] = sentiment_scores['compound']
    features['sentiment_positive'] = sentiment_scores['pos']
    features['sentiment_negative'] = sentiment_scores['neg']
    features['sentiment_neutral'] = sentiment_scores['neu']
    
    # Sentiment contradiction (positive words + negative sentiment = potential sarcasm)
    blob = TextBlob(text)
    features['sentiment_polarity'] = blob.sentiment.polarity
    features['sentiment_subjectivity'] = blob.sentiment.subjectivity
    
    # 2. INCONGRUITY PATTERNS
    # Detect contradictory sentiment words in same text
    positive_words = ['great', 'wonderful', 'amazing', 'perfect', 'brilliant', 'fantastic', 'excellent']
    negative_words = ['terrible', 'awful', 'horrible', 'worst', 'hate', 'stupid', 'ridiculous']
    
    text_lower = text.lower()
    pos_count = sum(1 for word in positive_words if word in text_lower)
    neg_count = sum(1 for word in negative_words if word in text_lower)
    
    features['positive_words'] = pos_count
    features['negative_words'] = neg_count
    features['sentiment_incongruity'] = 1 if (pos_count > 0 and neg_count > 0) else 0
    
    # 3. HYPERBOLE DETECTION
    # Excessive punctuation patterns
    features['exclamation_count'] = text.count('!')
    features['question_count'] = text.count('?')
    features['caps_ratio'] = sum(1 for c in text if c.isupper()) / (len(text) + 1)
    
    # Intensifier words
    intensifiers = ['so', 'very', 'extremely', 'absolutely', 'totally', 'completely', 'really']
    features['intensifier_count'] = sum(1 for word in intensifiers if word in text_lower)
    
    # 4. LEXICAL PATTERN SIGNALS
    # Text length and complexity
    words = text.split()
    features['word_count'] = len(words)
    features['char_count'] = len(text)
    features['avg_word_length'] = np.mean([len(word) for word in words]) if words else 0
    
    # Quotation marks (often used in sarcastic contexts)
    features['quote_count'] = text.count('"') + text.count("'")
    
    # Ellipsis usage
    features['ellipsis_count'] = text.count('...')
    
    return features

# Apply cleaning and feature extraction to Headlines dataset
print("1️⃣ Processing Headlines Dataset...")
headlines_df['cleaned_text'] = headlines_df['headline'].apply(clean_text)

# Extract contextual features for headlines
print("   📊 Extracting contextual features...")
headline_features = []
for idx, text in enumerate(headlines_df['cleaned_text']):
    if idx % 5000 == 0:
        print(f"      Processing headline {idx:,}/{len(headlines_df):,}")
    features = extract_contextual_features(text)
    headline_features.append(features)

# Convert to DataFrame
headline_features_df = pd.DataFrame(headline_features)
headline_final = pd.concat([headlines_df.reset_index(drop=True), headline_features_df], axis=1)

print(f"   ✅ Headlines processed: {len(headline_final)} samples")
print(f"   ✅ Features extracted: {len(headline_features_df.columns)} contextual features")

# Apply cleaning and feature extraction to Reddit dataset
print("\n2️⃣ Processing Reddit Dataset...")
reddit_sample['cleaned_text'] = reddit_sample['comment'].apply(clean_text)

print("   📊 Extracting contextual features...")
reddit_features = []
for idx, text in enumerate(reddit_sample['cleaned_text']):
    if idx % 10000 == 0:
        print(f"      Processing comment {idx:,}/{len(reddit_sample):,}")
    features = extract_contextual_features(text)
    reddit_features.append(features)

# Convert to DataFrame
reddit_features_df = pd.DataFrame(reddit_features)
reddit_final = pd.concat([reddit_sample.reset_index(drop=True), reddit_features_df], axis=1)

print(f"   ✅ Reddit processed: {len(reddit_final)} samples")
print(f"   ✅ Features extracted: {len(reddit_features_df.columns)} contextual features")

# Display feature summary
print(f"\n📊 CONTEXTUAL FEATURES EXTRACTED:")
print("=" * 35)
feature_categories = {
    'Sentiment': ['sentiment_compound', 'sentiment_polarity', 'sentiment_subjectivity'],
    'Incongruity': ['positive_words', 'negative_words', 'sentiment_incongruity'],
    'Hyperbole': ['exclamation_count', 'caps_ratio', 'intensifier_count'],
    'Lexical': ['word_count', 'avg_word_length', 'quote_count']
}

for category, features in feature_categories.items():
    print(f"✅ {category}: {', '.join(features)}")

print(f"\n🎯 Next: Combine datasets and create train/validation/test splits")

🧹 Applying Text Cleaning & Feature Extraction...
1️⃣ Processing Headlines Dataset...
   📊 Extracting contextual features...
      Processing headline 0/28,619
      Processing headline 5,000/28,619
      Processing headline 10,000/28,619
      Processing headline 15,000/28,619
      Processing headline 20,000/28,619
      Processing headline 25,000/28,619
   ✅ Headlines processed: 28619 samples
   ✅ Features extracted: 18 contextual features

2️⃣ Processing Reddit Dataset...
   📊 Extracting contextual features...
      Processing comment 0/50,000
      Processing comment 10,000/50,000
      Processing comment 20,000/50,000
      Processing comment 30,000/50,000
      Processing comment 40,000/50,000
   ✅ Reddit processed: 50000 samples
   ✅ Features extracted: 18 contextual features

📊 CONTEXTUAL FEATURES EXTRACTED:
✅ Sentiment: sentiment_compound, sentiment_polarity, sentiment_subjectivity
✅ Incongruity: positive_words, negative_words, sentiment_incongruity
✅ Hyperbole: exclamation_co

In [6]:
# Cell 3: Data Combination & Train/Validation/Test Splits
# Combine datasets and create stratified splits for model training

from sklearn.model_selection import train_test_split
import os

print("🔄 Combining Datasets & Creating Splits...")
print("=" * 45)

# Prepare datasets for combination
print("1️⃣ Preparing datasets for combination...")

# Headlines dataset preparation
headlines_processed = headline_final[['cleaned_text', 'is_sarcastic'] + list(headline_features_df.columns)].copy()
headlines_processed['dataset_source'] = 'headlines'
headlines_processed.rename(columns={'is_sarcastic': 'label'}, inplace=True)

# Reddit dataset preparation  
reddit_processed = reddit_final[['cleaned_text', 'label'] + list(reddit_features_df.columns)].copy()
reddit_processed['dataset_source'] = 'reddit'

print(f"   ✅ Headlines: {len(headlines_processed)} samples")
print(f"   ✅ Reddit: {len(reddit_processed)} samples")

# Combine datasets
print("\n2️⃣ Combining datasets...")
combined_data = pd.concat([headlines_processed, reddit_processed], ignore_index=True)
combined_data = combined_data.dropna(subset=['cleaned_text'])  # Remove any null texts

print(f"   ✅ Combined dataset: {len(combined_data):,} samples")
print(f"   ✅ Total sarcastic: {combined_data['label'].sum():,} ({combined_data['label'].mean():.1%})")
print(f"   ✅ Total non-sarcastic: {(combined_data['label'] == 0).sum():,}")

# Display feature columns
feature_columns = [col for col in combined_data.columns if col not in ['cleaned_text', 'label', 'dataset_source']]
print(f"   ✅ Feature columns: {len(feature_columns)}")

# Create stratified splits
print("\n3️⃣ Creating train/validation/test splits...")

# First split: 80% train, 20% temp (which will become 10% val, 10% test)
X = combined_data[['cleaned_text'] + feature_columns]
y = combined_data['label']

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Second split: Split the 20% into 10% validation, 10% test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"   📊 Train set: {len(X_train):,} samples ({y_train.mean():.1%} sarcastic)")
print(f"   📊 Validation set: {len(X_val):,} samples ({y_val.mean():.1%} sarcastic)")
print(f"   📊 Test set: {len(X_test):,} samples ({y_test.mean():.1%} sarcastic)")

# Save preprocessed data
print("\n4️⃣ Saving preprocessed data...")

# Create processed data directory
os.makedirs('data/processed', exist_ok=True)

# Save splits as separate files
train_data = pd.concat([X_train, y_train], axis=1)
val_data = pd.concat([X_val, y_val], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

train_data.to_csv('data/processed/train_data.csv', index=False)
val_data.to_csv('data/processed/val_data.csv', index=False)
test_data.to_csv('data/processed/test_data.csv', index=False)

print(f"   ✅ Saved train_data.csv: {len(train_data):,} samples")
print(f"   ✅ Saved val_data.csv: {len(val_data):,} samples")
print(f"   ✅ Saved test_data.csv: {len(test_data):,} samples")

# Save feature information for model training
feature_info = {
    'feature_columns': feature_columns,
    'text_column': 'cleaned_text',
    'label_column': 'label',
    'total_samples': len(combined_data),
    'sarcasm_rate': float(combined_data['label'].mean()),
    'train_size': len(train_data),
    'val_size': len(val_data),
    'test_size': len(test_data)
}

with open('data/processed/feature_info.json', 'w') as f:
    json.dump(feature_info, f, indent=2)

print(f"   ✅ Saved feature_info.json")

# Display sample of processed data
print(f"\n📋 SAMPLE OF PROCESSED DATA:")
print("=" * 32)
sample_data = train_data.head(3)
for idx, row in sample_data.iterrows():
    label = "SARCASTIC" if row['label'] else "NORMAL"
    text = row['cleaned_text'][:60] + "..." if len(row['cleaned_text']) > 60 else row['cleaned_text']
    sentiment = row['sentiment_compound']
    print(f"[{label}] {text}")
    print(f"         Sentiment: {sentiment:.3f}, Words: {row['positive_words']}+/{row['negative_words']}-")

print(f"\n🎯 PREPROCESSING COMPLETE!")
print("=" * 30)
print("✅ Text cleaning applied")
print("✅ Contextual features extracted") 
print("✅ Train/val/test splits created")
print("✅ Data saved for model training")
print(f"\n📂 Files saved in: data/processed/")
print("   - train_data.csv")
print("   - val_data.csv") 
print("   - test_data.csv")
print("   - feature_info.json")

print(f"\n🚀 Ready for Phase 2: Model Development!")

🔄 Combining Datasets & Creating Splits...
1️⃣ Preparing datasets for combination...
   ✅ Headlines: 28619 samples
   ✅ Reddit: 50000 samples

2️⃣ Combining datasets...
   ✅ Combined dataset: 78,619 samples
   ✅ Total sarcastic: 38,696 (49.2%)
   ✅ Total non-sarcastic: 39,923
   ✅ Feature columns: 18

3️⃣ Creating train/validation/test splits...
   📊 Train set: 62,895 samples (49.2% sarcastic)
   📊 Validation set: 7,862 samples (49.2% sarcastic)
   📊 Test set: 7,862 samples (49.2% sarcastic)

4️⃣ Saving preprocessed data...
   ✅ Saved train_data.csv: 62,895 samples
   ✅ Saved val_data.csv: 7,862 samples
   ✅ Saved test_data.csv: 7,862 samples
   ✅ Saved feature_info.json

📋 SAMPLE OF PROCESSED DATA:
[NORMAL] amazing couple adopts 7 siblings to keep them together
         Sentiment: 0.670, Words: 1+/0-
[SARCASTIC] 92% of area woman's holiday recipes involve pulverizing bag ...
         Sentiment: 0.402, Words: 0+/0-
[SARCASTIC] that's likely the case, so i will blame f.o.o.l for not call